In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
SEED = 42
def cv_loop_acc(X, y, model, N):
    mean_acc = 0.0
    for i in range(0,N,1):
        X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(
                                       X, y, test_size=1.0/float(N), 
                                       random_state = i*SEED)
        model.fit(X_train, y_train)
        preds = model.predict(X_cv)
        acc = metrics.accuracy_score(y_cv, preds)
        #print acc
        mean_acc += acc
    return mean_acc/N

In [3]:
SEED = 42
def cv_loop_auc(X, y, model, N):
    mean_auc = 0.0
    for i in range(0,N,1):
        X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(
                                       X, y, test_size=1.0/float(N), 
                                       random_state = i*SEED)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_cv)[:,1]
        auc = metrics.roc_auc_score(y_cv, preds)
        #print auc
        mean_auc += auc
    return mean_auc/N

In [4]:
SEED = 42
def latest_day_loop_acc(X_train,y_train,X_test,y_test,model):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = metrics.accuracy_score(y_test, preds)
    return acc

In [5]:
def read_csv(day_trade):
    data_up = []
    data_down = []
    path = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/Train_Test_Data/ML_data_2014'
    for j,i in enumerate(day_trade):
        for k in range(0,len(i),1):
            path_up = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'UP' + '.csv'
            path_down = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'DOWN' + '.csv'
            data_up.append(pd.read_csv(path_up))
            data_down.append(pd.read_csv(path_down))
            #print path_down
    return data_up,data_down

In [6]:
day_trade = [[2,3,6,7,8,10,13,14,15,16,17,20,21,22,23,24,27,28,29,30],\
             [7,10,11,12,13,17,18,19,21,24,25,26,27],\
             [3,4,5,6,7,10,11,13,14,17,18,19,20,24,25,26,27,31]]

In [7]:
data_2014_up, data_2014_down = read_csv(day_trade)

In [11]:
from sklearn.grid_search import (GridSearchCV, RandomizedSearchCV)
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier)
from sklearn import (metrics, cross_validation, linear_model, preprocessing)

In [43]:
latest_min = 60 * 30
pred_sec = 10
Accuracy_Mean_Day = []
Accuracy_Std_Day = []
Feature_Importance = []
Feature_Importance_Day_one = []
Feature_Importance_Day_two = []
Feature_Importance_Day_three = []
Feature_Importance_Day_four = []
Feature_Importance_Day_five = []


for day in range(0,10,1):
    Accuracy = []
    Feature_Importance_one = []
    Feature_Importance_two = []
    Feature_Importance_three = []
    Feature_Importance_four = []
    Feature_Importance_five = []
    for i in range(0,9000-latest_min-pred_sec,pred_sec):

        #print '----------------------------Rolling Window Time = %s----------------------------'%(i/5.0)
        data_train = data_2014_up[day][i:i+latest_min]
        X_train = data_train.drop(["0"],axis=1)
        y_train = data_train['0']

        data_test = data_2014_up[day][i+latest_min:i+latest_min+pred_sec]
        X_test = data_test.drop(["0"],axis=1)
        y_test = data_test['0']

        # hyper-parameter
        num_trees = [10]
        max_depth = [10]
        criterion = ['entropy']
        min_samples_leaf = [5]
        min_samples_split = [2]
        max_features = [None]

        # cv
        model_grid = {'max_features':max_features,'n_estimators':num_trees,'max_depth':max_depth,\
                      'min_samples_split':min_samples_split,'criterion':criterion,\
                      'min_samples_leaf':min_samples_leaf}
        
        model = RandomForestClassifier(random_state = 0)
        Grid = GridSearchCV(model, model_grid, cv = 5)
        Grid.fit(X_train,y_train) 
        #print Grid.best_params_
        #print 'CV Best Score = %s'%(Grid.best_score_)

        # training & testing
        model.set_params(**Grid.best_params_)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        acc = metrics.accuracy_score(y_test, predictions)
        #print 'Accuracy = %s'%(acc)
        Accuracy.append(acc)
        feature_imp = dict(zip([i for i in range(0,64,1)],model.feature_importances_))
        
        Top_five = sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:5]
        #print 'Top five importance features = %s'%(Top_five)
        
        Feature_Importance_one.append(Top_five[0][0])
        Feature_Importance_two.append(Top_five[1][0])
        Feature_Importance_three.append(Top_five[2][0])
        Feature_Importance_four.append(Top_five[3][0])
        Feature_Importance_five.append(Top_five[4][0])
        
    print 'Day = %s, mean = %s, std = %s'%(day,np.mean(Accuracy),np.std(Accuracy))
    Accuracy_Mean_Day.append(Accuracy)
    Accuracy_Std_Day.append(Accuracy)
    Feature_Importance_Day_one.append(Feature_Importance_one)
    Feature_Importance_Day_two.append(Feature_Importance_two)
    Feature_Importance_Day_three.append(Feature_Importance_three)
    Feature_Importance_Day_four.append(Feature_Importance_four)
    Feature_Importance_Day_five.append(Feature_Importance_five)
    

Day = 0, mean = 0.907649513213, std = 0.224593366683
Day = 1, mean = 0.930876216968, std = 0.193045095452


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Day = 2, mean = 0.944645340751, std = 0.177515242786
Day = 3, mean = 0.895966620306, std = 0.240185090887
Day = 4, mean = 0.928511821975, std = 0.19862359356
Day = 5, mean = 0.895271210014, std = 0.236819215668
Day = 6, mean = 0.906258692629, std = 0.226793074378
Day = 7, mean = 0.91668984701, std = 0.208176805926
Day = 8, mean = 0.922670375522, std = 0.201180358605
Day = 9, mean = 0.915299026426, std = 0.213819203746


In [62]:
np.array(Accuracy_Mean_Day).mean(axis=1),np.array(Accuracy_Mean_Day).std(axis=1)

(array([ 0.90764951,  0.93087622,  0.94464534,  0.89596662,  0.92851182,
         0.89527121,  0.90625869,  0.91668985,  0.92267038,  0.91529903]),
 array([ 0.22459337,  0.1930451 ,  0.17751524,  0.24018509,  0.19862359,
         0.23681922,  0.22679307,  0.20817681,  0.20118036,  0.2138192 ]))

In [65]:
mean(np.array(Accuracy_Mean_Day).mean(axis=1))

0.91638386648122394

In [21]:
i = 0
day = 0
latest_min = 60 * 30
pred_sec = 5

data_train = data_2014_up[day][i:i+latest_min]
X_train = data_train.drop(["0"],axis=1)
y_train = data_train['0']
data_test = data_2014_up[day][i+latest_min:i+latest_min+pred_sec]
X_test = data_test.drop(["0"],axis=1)
y_test = data_test['0']

In [22]:
Feature_data = X_train
Label = y_train

In [24]:
num_trees = [5,10,15,20,25]
max_depth = [10,15,20.25]
criterion = ['entropy']
min_samples_leaf = [5,10,15]
min_samples_split = [2,3]
max_features = [None]

In [57]:
model_grid = {'max_features':max_features,'n_estimators':num_trees,'max_depth':max_depth,\
              'min_samples_split':min_samples_split,'criterion':criterion,\
              'min_samples_leaf':min_samples_leaf}
model1 = RandomForestClassifier(random_state = 0)
Grid = GridSearchCV(model1, model_grid, cv = 10)#, scoring = 'acc')
%time Grid.fit(Feature_data, Label) 
Grid.grid_scores_

KeyboardInterrupt: 

In [27]:
Grid.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': None,
 'min_samples_leaf': 15,
 'min_samples_split': 2,
 'n_estimators': 10}

In [28]:
model1.set_params(**Grid.best_params_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=15,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [29]:
y = y_train
X = X_train
%time model1.fit(X, y)
predictions = model1.predict(X_test)
#model1.predict_proba(X_test)[:,1]

CPU times: user 216 ms, sys: 16 µs, total: 216 ms
Wall time: 271 ms


In [30]:
predictions,y_test

(array([ 1.,  1.,  1.,  1.,  1.]), 1800    1.0
 1801    1.0
 1802    1.0
 1803    1.0
 1804    1.0
 Name: 0, dtype: float64)

In [31]:
metrics.accuracy_score(y_test, predictions)

1.0

In [80]:
feature_imp = dict(zip([i for i in range(0,64,1)],model1.feature_importances_))

NotFittedError: Estimator not fitted, call `fit` before `feature_importances_`.

In [86]:
print ' = %s'%(sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:10])

TypeError: not all arguments converted during string formatting

In [121]:
from sklearn.metrics import confusion_matrix

In [122]:
confusion_matrix(y_test,predictions)

array([[5]])

In [125]:
pd.crosstab(y_test,predictions,rownames = ['actual'],colnames=['preds'])

preds,1.0
actual,
1.0,5


In [26]:
from sklearn.grid_search import (GridSearchCV, RandomizedSearchCV)
from sklearn.ensemble import RandomForestClassifier

Feature_data = data_train.drop(["0"],axis=1)
Label = data_train['0']

num_trees = [10]
max_depth = [None]
criterion = ['entropy']
min_samples_leaf = [1,3,5,7]
min_samples_split = [3,5]
max_features = [None]

model_grid = {'max_features':max_features,'n_estimators':num_trees,'max_depth':max_depth,\
              'min_samples_split':min_samples_split,'criterion':criterion,\
              'min_samples_leaf':min_samples_leaf}

model1 = RandomForestClassifier(random_state = 0)
Grid = GridSearchCV(model1, model_grid, cv = 5, scoring = 'roc_auc')

%time Grid.fit(Feature_data, Label) 
Grid.grid_scores_

CPU times: user 7.25 s, sys: 0 ns, total: 7.25 s
Wall time: 7.85 s


[mean: 0.75950, std: 0.12004, params: {'min_samples_leaf': 1, 'n_estimators': 10, 'min_samples_split': 3, 'criterion': 'entropy', 'max_features': None, 'max_depth': None},
 mean: 0.75244, std: 0.11471, params: {'min_samples_leaf': 1, 'n_estimators': 10, 'min_samples_split': 5, 'criterion': 'entropy', 'max_features': None, 'max_depth': None},
 mean: 0.74369, std: 0.16452, params: {'min_samples_leaf': 3, 'n_estimators': 10, 'min_samples_split': 3, 'criterion': 'entropy', 'max_features': None, 'max_depth': None},
 mean: 0.74369, std: 0.16452, params: {'min_samples_leaf': 3, 'n_estimators': 10, 'min_samples_split': 5, 'criterion': 'entropy', 'max_features': None, 'max_depth': None},
 mean: 0.75741, std: 0.15673, params: {'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 3, 'criterion': 'entropy', 'max_features': None, 'max_depth': None},
 mean: 0.75741, std: 0.15673, params: {'min_samples_leaf': 5, 'n_estimators': 10, 'min_samples_split': 5, 'criterion': 'entropy', 'max_featu

In [93]:
Grid.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'max_features': None,
 'min_samples_leaf': 5,
 'min_samples_split': 3,
 'n_estimators': 10}